<br><br><center><h1 style="font-size:2.7em;color:#000">MACHINE LEARNING LAB</h1></center>
<br><center><h1 style="font-size:2em;color:#2467C0">MINIPROJECT - PYTORCH</h1></center>
<br>
<h1 style="font-size:2em;color:#000">Juan Fernando Espinosa</h1>
<h1 style="font-size:2em;color:#000">303158</h1>

---
<h3 style="font-size:2em;color:#ff4411">1. DATA PRE-PROCESSING</h3>

In [1]:
import torch
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import pandas as pd
import numpy as np
from collections import Counter
import math
import matplotlib.pyplot as plt

In [2]:
missing_values = ['-','na','Nan','nan','n/a','?']

Wine = pd.read_csv("winequality-white.csv", sep=';', na_values = missing_values)
Wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


<h5 style="font-size:1.5em;color:#2467C0">Data normalization</h5>

----

In [3]:
def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
    dataNorm["quality"]=dataset["quality"]
    return dataNorm
Wine = normalize(Wine)
Wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.307692,0.186275,0.216867,0.308282,0.106825,0.149826,0.373550,0.267785,0.254545,0.267442,0.129032,6
1,0.240385,0.215686,0.204819,0.015337,0.118694,0.041812,0.285383,0.132832,0.527273,0.313953,0.241935,6
2,0.413462,0.196078,0.240964,0.096626,0.121662,0.097561,0.204176,0.154039,0.490909,0.255814,0.338710,6
3,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,6
4,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,6


<h5 style="font-size:1.5em;color:#2467C0">Check number of classes in the dataset</h5>

----

In [4]:
test_classes = Wine.quality
output = len(set(test_classes))
print('Number of classes:', output)

Number of classes: 7


<h3 style="font-size:2em;color:#ff4411">2. INITIALIZATION OF THE REGRESSION</h3>

<h5 style="font-size:1.5em;color:#2467C0">Partition of the dataset</h5>

----

In [5]:
y = Wine['quality'].values
X =  Wine.drop(['quality'], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train.shape, y_train.shape

((3428, 11), (3428,))

<h5 style="font-size:1.5em;color:#2467C0">Initialization of the regression and the layer structure</h5>
<p>Considering the given info: input / 3 FC layers / output.</p>

----

In [6]:
network = torch.nn.Sequential(
        torch.nn.Linear(11, 64), # input
        torch.nn.Sigmoid(),     
        torch.nn.Linear(64, 64),# First hidden layer
        torch.nn.Sigmoid(), 
        torch.nn.Linear(64, 64),# Second hidden layer
        torch.nn.Sigmoid(),
        torch.nn.Linear(64, 64),# Third hidden layer
        torch.nn.Sigmoid(),
        torch.nn.Linear(64, 1),
    )
optimizer = torch.optim.Adam(network.parameters(), lr=0.01)
loss = torch.nn.MSELoss()

batch_sizeT = 5
epochs = 200

# Transform Numpy array to Tensor variables.
X = torch.from_numpy(X_train)
print(X.dtype)

y = torch.from_numpy(y_train)
print(y.dtype)
datasets = torch.utils.data.TensorDataset(X.float(), y.long)
loader = Data.DataLoader(
    dataset=datasets, 
    batch_size = batch_sizeT, 
    shuffle=True, num_workers=0,)
plt.ion()
loss_array = []
for epoch in range(epochs):
    
    for step, (batch_X, batch_y) in enumerate(loader): # for each training step
        X_temporal = Variable(batch_X)
        y_temporal = Variable(batch_y)
        prediction = network(X_temporal.float())     # input x and predict based on x
        losse = loss(prediction, y_temporal.float())
        loss_array.append(losse.data)


        optimizer.zero_grad()  # It is required to clear the gradients
        losse.backward()         # backpropagation
        optimizer.step()
    print(f"{epoch+1} epoch | loss = {losse}")
plt.plot(loss_array)

torch.float64
torch.int64


AttributeError: 'builtin_function_or_method' object has no attribute 'size'

In [ ]:
X_testing = torch.from_numpy(X_test)
y_testing = torch.from_numpy(y_test)
prediction = network(X_testing.float())
losse = loss(prediction, y_testing.float())


print("The test loss is {:.2}".format(losse.data.item()))



<h3 style="font-size:1.5em;color:#6e00db">Observations</h3> 

 ----
 <ol>
    <li>Although the RMSE of the prediction is <strong>low</strong> it lacks to predict correctly the quality for which each wine belongs.</li><br>
    <li>By searching for a solution of why the behavior is poor the description of the target column "quality" explicitly mentioned that the values are based on <strong>sensory data</strong> which could be named as perception results, therefore the prediction is not easy to execute.</li><br>
    <li>It is always good practice to execute <strong>cross-validation</strong> for all hyperparameters. While executing the model it is not possible to have certainty of which combination of nodes in hidden layers, batch size, etc., is the optimal.</li><br>
    <li>Moreover, for curiosity, I tested the model with different activation functions. <strong>Leaky ReLu</strong> returns a RMSE around 0.7 (Cross-validated | hyperparameters: batch size, # nodes in hidden layers) which is slightly lower in comparison with the Sigmoid function presented in the model above. Additionally, the convergence of the loss is better. (check 4. Tests)</li><br>
    
</ol>

<h3 style="font-size:2em;color:#ff4411">3. BIBLIOGRAPHY</h3>

 <ol>
    <li>P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.</li><br>
    <li>Phillips, B. (2018, December 15). Simple Regression with Neural Networks in PyTorch. Retrieved from https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379.</li><br>
    <li>Tsvetkov, V. (2019, September 17). A comprehensive intro to PyTorch. Retrieved from https://blog.tensorpad.com/a-comprehensive-intro-to-pytorch/.</li><br>
    
</ol>

<h3 style="font-size:2em;color:#ff4411">4. TESTS</h3>

<h5 style="font-size:1.5em;color:#2467C0">Leaky-ReLu as activation function</h5>

In [ ]:
network = torch.nn.Sequential(
        torch.nn.Linear(11, 150), # input
        torch.nn.LeakyReLU(),     
        torch.nn.Linear(150, 150),# First hidden layer
        torch.nn.LeakyReLU(), 
        torch.nn.Linear(150, 150),# Second hidden layer
        torch.nn.LeakyReLU(),
        torch.nn.Linear(150, 150),# Third hidden layer
        torch.nn.LeakyReLU(),
        torch.nn.Linear(150, 1),
    )
optimizer = torch.optim.Adam(network.parameters(), lr=0.01)
loss = torch.nn.MSELoss()

batch_sizeT = 64
epochs = 100

# Transform Numpy array to Tensor variables.
X = torch.from_numpy(X_train)
y = torch.from_numpy(y_train)

datasets = torch.utils.data.TensorDataset(X, y)

loader = Data.DataLoader(
    dataset=datasets, 
    batch_size = batch_sizeT, 
    shuffle=True, num_workers=0,)
plt.ion()
loss_array = []
for epoch in range(epochs):
    
    for step, (batch_X, batch_y) in enumerate(loader): # for each training step
        X_temporal = Variable(batch_X)
        y_temporal = Variable(batch_y)
        prediction = network(X_temporal.float())     # input x and predict based on x
        losse = loss(prediction, y_temporal.float())
        loss_array.append(losse.data)


        optimizer.zero_grad()  # It is required to clear the gradients
        losse.backward()         # backpropagation
        optimizer.step()
    print(f"{epoch+1} epoch | loss = {losse}")
plt.plot(loss_array)

In [ ]:
loss_test = []
X_testing = torch.from_numpy(X_test)
y_testing = torch.from_numpy(y_test)
prediction = network(X_testing.float())
losse = loss(prediction, y_testing.float())



print("The test loss is {:.2}".format(losse.data.item()))
